In [ ]:
from pathlib import Path
import pandas as pd
pd.set_option('display.max_columns', None)
from tqdm import tqdm_pandas,tqdm
tqdm.pandas()
from Bio.PDB.Polypeptide import protein_letters_3to1,protein_letters_3to1_extended
import urllib.request
import re
import swifter
from data_processing.clinvar import clinvar_pathogenicity_status, is_protein_variant,categorize_snv,parse_clinvar_name

In [ ]:
clinvar_variant_summary = pd.read_pickle("/mnt/d/mave_calibration/cache/variant_summary_2024-08.pkl")

In [ ]:
clinvar_variant_summary.shape

In [ ]:
snv = clinvar_variant_summary[clinvar_variant_summary["Type"] == "single nucleotide variant"]

In [ ]:
snv = snv.assign(snv_category = snv.Name.progress_apply(categorize_snv))

In [ ]:
snv.snv_category.value_counts()

In [ ]:
snv = snv.assign(
    is_pathogenic = (snv.ClinicalSignificance.isin({"Pathogenic","Likely pathogenic", "Pathogenic/Likely pathogenic"})) & \
        (~snv.ReviewStatus.isin({'no clasification provided', 'no assertion criteria provided','no classification for the single variant'})),
    is_benign = (snv.ClinicalSignificance.isin({"Benign", "Likely benign", "Benign/Likely benign"})) & \
        (~snv.ReviewStatus.isin({'no clasification provided', 'no assertion criteria provided','no classification for the single variant'}))
)

In [ ]:
snv.GeneID = snv.GeneID.astype(str)

In [ ]:
snv.shape

In [ ]:
parsed_snv_names = snv.Name.progress_apply(parse_clinvar_name)

In [ ]:
len(parsed_snv_names)

In [ ]:
parsed_name_df = pd.DataFrame.from_records(parsed_snv_names.values,index=snv.index)

In [ ]:
parsed_name_df.shape

need to use the same index in concat!

In [ ]:
snv_expanded = pd.concat((snv,parsed_name_df),axis=1)
if (snv_expanded.Name != snv_expanded.name).any():
    raise ValueError("Name and name do not match")
snv_expanded.drop(columns=["name"],inplace=True)

In [ ]:
snv_expanded.head()

In [2]:
from data_processing.clinvar import getClinvar
import pandas as pd
pd.set_option('display.max_columns', None)

In [3]:
clinvar = getClinvar(year="2024",month="08",
                    use_cached_processed_file=True,
                    cache_dir="/mnt/d/mave_calibration/cache",)

In [5]:
clinvar.snv_category.value_counts()

snv_category
missense                 2889704
silent                   1213559
non_protein_variant      1099086
nonsense                  134097
unknown                       18
other_protein_variant          2
Name: count, dtype: int64

In [10]:
clinvar.loc[(clinvar.is_pathogenic),'snv_category'].value_counts()

snv_category
nonsense               105266
missense                94891
non_protein_variant     66539
silent                   1092
unknown                    14
Name: count, dtype: int64

In [11]:
clinvar.ClinicalSignificance.value_counts()

ClinicalSignificance
Uncertain significance                          2630778
Likely benign                                   1682950
Benign                                           353039
Conflicting classifications of pathogenicity     241561
Pathogenic                                       151569
                                                 ...   
Likely pathogenic; Affects                            2
Likely pathogenic; protective                         2
Likely benign; association                            2
Likely benign; risk factor                            2
confers sensitivity; other                            2
Name: count, Length: 86, dtype: int64

In [1]:
clinvar[(clinvar.geneSymbol == "BRCA2") & \
        (clinvar.snv_category == "missense") & \
            (clinvar.is_pathogenic == True)]

NameError: name 'clinvar' is not defined